<a href="https://colab.research.google.com/github/R-Madhuram/DartmouthCapstone_CrudeOilPricePrediction/blob/main/notebooks/4_LinkedIN_Silverkite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries 


In [ ]:
# Mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product


/content/drive/MyDrive/Time Series Forecasting Product


In [ ]:
#Install Greykite
!pip install greykite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for osqp: filename=osqp-0.6.1-cp39-cp39-linux_x86_64.whl size=209925 sha256=73ad601bb90ec30cab797896b19a8e63e0dfd520b7ddff719cf4629dbd88225f
  Stored in directory: /root/.cache/pip/wheels/3d/34/b7/b8b2af435e6e6d4ce9c92611064e6c8589579bbf4148ea4d50
Successfully built osqp
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstall

In [ ]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from greykite.framework.templates.autogen.forecast_config import *
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.common.features.timeseries_features import *
from greykite.common.evaluation import EvaluationMetricEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from plotly.offline import iplot


# Load Data

In [ ]:
# load data
us_cushing = pd.read_csv('/content/drive/MyDrive/PCDS_OilPricePrediction/data/USCushing.csv')
eu_brent = pd.read_csv('/content/drive/MyDrive/PCDS_OilPricePrediction/data/EuropeanBrent.csv')


# 1_Modeling with LinkedIn Silverkite


## Cushing Data

In [ ]:
# Read data 
us_cushing.head().append(us_cushing.tail())

,Date,"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"
0,"Jan 02, 1986",25.56
1,"Jan 03, 1986",26.00
2,"Jan 06, 1986",26.53
3,"Jan 07, 1986",25.85
4,"Jan 08, 1986",25.87
9357,"Feb 22, 2023",73.95
9358,"Feb 23, 2023",75.26
9359,"Feb 24, 2023",76.19
9360,"Feb 27, 2023",75.57
9361,NaN,NaN


In [ ]:
# Renaming columns
df_cushing_fp = us_cushing.rename(columns = {'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)':'y'})
df_cushing_fp.head(3).append(df_cushing_fp.tail(3))

,Date,y
0,"Jan 02, 1986",25.56
1,"Jan 03, 1986",26.00
2,"Jan 06, 1986",26.53
9359,"Feb 24, 2023",76.19
9360,"Feb 27, 2023",75.57
9361,NaN,NaN


### Silverkite Preparations

In [ ]:
# Specifying time series names/variables
"""Here we specify the date column,y column, the frequency 
and until when is training data. It specifies the skeleton of the data."""

metadata = MetadataParam(time_col = 'Date',
                         value_col='y',
                         freq = 'B',
                         train_end_date = pd.to_datetime('2023-2-27'))
metadata

MetadataParam(anomaly_info=None, date_format=None, freq='B', time_col='Date', train_end_date=Timestamp('2023-02-27 00:00:00'), value_col='y')

In [ ]:
# Growth term possibilities
"""Silverkite and all other models till now we built can work with dictionaries"""
"""we can specify everything step by step but it is prone to spelling mistakes"""
growth = dict(growth_term = ['linear'])
print(growth)
print(type(growth))

{'growth_term': ['linear']}
<class 'dict'>


In [ ]:
# seasonality term possibilities
""" In sarimax we had only one seasonality, in prophet we had 3 seasonality namely
daily, weekly and yearly and we could have all of them. Assuming we have hourly data
we can have upto 5 seasonalities"""
"""yearly_seasonality and rest all can be set to :none,true,auto"""

seasonality = dict(yearly_seasonality = 'auto',
                   monthly_seasonality = 'auto',
                   weekly_seasonality = 'auto')
print(seasonality)
print(type(seasonality))

{'yearly_seasonality': 'auto', 'monthly_seasonality': 'auto', 'weekly_seasonality': 'auto'}
<class 'dict'>


In [ ]:
# Changepoints --> reflects changes in trends
changepoints = dict(changepoints_dict = dict(method = 'auto'))

In [ ]:
# Auto Regression --> dependednt on the forecasting horizon
autoregression = dict(autoreg_dict = 'auto')
autoregression

{'autoreg_dict': 'auto'}

Silverkite offers the wrapper or fitting algorithms strategy that takes in to account all the possibilities that we have created and combines them.

Linear regression, elastic net, ridge, lasso, stochastic gradient decent, lars, lasso lars, random forest, gradient boosting are some of the algorithm fitting possibilities.

LR is poor with collinearity, stochastic gradient decent is unstable, lars is sensitive to outliers or noise, random forest and gradient boosting i.e tree models do not model the trend well. 

We will try Linear regression, Ridge regression, Random forest and gradient boosting.

Ridge regression aims to decrease overfittign and not to have independent variables with too high importance/relevance.





In [ ]:
# Custom fit Algorithm
custom = dict(fit_algorithm_dict = [dict(fit_algorithm = 'gradient_boosting'),
                                    dict(fit_algorithm = 'ridge')])

### 1) Silverkite Model

In [ ]:
#Build the model
model_components = ModelComponentsParam(growth = growth,
                                        seasonality = seasonality,
                                        changepoints = changepoints,
                                        autoregression = autoregression,
                                        custom = custom)

In [ ]:
#Cross-validation
evaluation_period = EvaluationPeriodParam(cv_min_train_periods= 506,
                                          cv_expanding_window = False,
                                          cv_max_splits = None,
                                          cv_periods_between_splits = 22,
                                          cv_horizon = 22)

In [ ]:
#Evaluation metric 
evaluation_metric = EvaluationMetricParam(
    cv_selection_metric = EvaluationMetricEnum.RootMeanSquaredError.name)

In [ ]:
#Configuration
config = ForecastConfig(model_template = ModelTemplateEnum.SILVERKITE.name,
                        forecast_horizon = 289,
                        metadata_param = metadata,
                        model_components_param = model_components,
                        evaluation_period_param=evaluation_period,
                        evaluation_metric_param = evaluation_metric)

## Brent Data

In [ ]:
# Read data 
eu_brent.head().append(eu_brent.tail())

,Date,Europe Brent Spot Price FOB (Dollars per Barrel)
0,"May 20, 1987",18.63
1,"May 21, 1987",18.45
2,"May 22, 1987",18.55
3,"May 25, 1987",18.60
4,"May 26, 1987",18.63
9077,"Feb 22, 2023",79.55
9078,"Feb 23, 2023",82.08
9079,"Feb 24, 2023",82.31
9080,"Feb 27, 2023",82.31
9081,NaN,NaN


In [ ]:
# Renaming columns
eu_brent_fp = eu_brent.rename(columns = {'Europe Brent Spot Price FOB (Dollars per Barrel)':'y'})
eu_brent_fp.head(3).append(eu_brent_fp.tail(3))

,Date,y
0,"May 20, 1987",18.63
1,"May 21, 1987",18.45
2,"May 22, 1987",18.55
9079,"Feb 24, 2023",82.31
9080,"Feb 27, 2023",82.31
9081,NaN,NaN


### Silverkite Preparations

In [ ]:
# Specifying time series names/variables
"""Here we specify the date column,y column, the frequency 
and until when is training data. It specifies the skeleton of the data."""

metadata_brent = MetadataParam(time_col = 'Date',
                         value_col='y',
                         freq = 'D',
                         train_end_date = pd.to_datetime('2023-2-27'))
metadata_brent

MetadataParam(anomaly_info=None, date_format=None, freq='D', time_col='Date', train_end_date=Timestamp('2023-02-27 00:00:00'), value_col='y')

In [ ]:
# Growth term possibilities
"""Silverkite and all other models till now we built can work with dictionaries"""
"""we can specify everything step by step but it is prone to spelling mistakes"""
growth_brent = dict(growth_term = ['linear'])
print(growth)
print(type(growth))

{'growth_term': ['linear']}
<class 'dict'>


In [ ]:
# seasonality term possibilities
""" In sarimax we had only one seasonality, in prophet we had 3 seasonality namely
daily, weekly and yearly and we could have all of them. Assuming we have hourly data
we can have upto 5 seasonalities"""
"""yearly_seasonality and rest all can be set to :none,true,auto"""

seasonality_brent = dict(yearly_seasonality = 'auto',
                   monthly_seasonality = 'auto',
                   weekly_seasonality = 'auto')
print(seasonality_brent)
print(type(seasonality_brent))

{'yearly_seasonality': 'auto', 'monthly_seasonality': 'auto', 'weekly_seasonality': 'auto'}
<class 'dict'>


In [ ]:
# Changepoints --> reflects changes in trends
changepoints_brent = dict(changepoints_dict = dict(method = 'auto'))

In [ ]:
# Auto Regression --> dependednt on the forecasting horizon
autoregression_brent = dict(autoreg_dict = 'auto')
autoregression

{'autoreg_dict': 'auto'}

### 1) Silverkite Model

In [ ]:
# Custom fit Algorithm
custom_brent = dict(fit_algorithm_dict = [dict(fit_algorithm = 'gradient_boosting'),
                                    dict(fit_algorithm = 'ridge')])

In [ ]:
#Build the model
model_components_brent = ModelComponentsParam(growth = growth_brent,
                                        seasonality = seasonality_brent,
                                        changepoints = changepoints_brent,
                                        autoregression = autoregression_brent,
                                        custom = custom_brent)

In [ ]:
#Cross-validation
evaluation_period_brent = EvaluationPeriodParam(cv_min_train_periods= 506,
                                          cv_expanding_window = False,
                                          cv_max_splits = None,
                                          cv_periods_between_splits = 22,
                                          cv_horizon = 22)

In [ ]:
#Evaluation metric 
evaluation_metric_brent = EvaluationMetricParam(
    cv_selection_metric = EvaluationMetricEnum.RootMeanSquaredError.name)

In [ ]:
#Configuration
config_brent = ForecastConfig(model_template = ModelTemplateEnum.SILVERKITE.name,
                        forecast_horizon = 289,
                        metadata_param = metadata_brent,
                        model_components_param = model_components_brent,
                        evaluation_period_param=evaluation_period_brent,
                        evaluation_metric_param = evaluation_metric_brent)

# 2_Forecasting

## Cushing Data

### 1) Silverkite Model

In [ ]:
#Forecasting
forecaster = Forecaster()
result = forecaster.run_forecast_config(df = df_cushing_fp,
                                        config = config)

/usr/local/lib/python3.9/dist-packages/greykite/common/time_properties.py:197: UserWarning:

Dropped 1 dates when filling gaps in input data. Provide data frequency and make sure data points are evenly spaced.



Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.9/dist-packages/greykite/common/evaluation.py:153: UserWarning:

9 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.9/dist-packages/greykite/common/evaluation.py:153: UserWarning:

9 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.9/dist-packages/greykite/common/evaluation.py:153: UserWarning:

9 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.9/dist-packages/greykite/common/evaluation.py:153: UserWarning:

9 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.9/dist-packages/greykite/common/evaluation.py:153: UserWarning:

9 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.9/dist-packages/greykite/common/evaluation.py:153: UserWarning:

9 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.9/dist-pack

In [ ]:
#Look at the model summary
summary = result.model[-1].summary() #-1 restrieves the estimator from the pipeline
print(summary)

================================ Model Summary =================================

Number of observations: 9693,   Number of features: 230
Method: Gradient Boosting
Number of Trees: 100,   Criterion: FRIEDMAN_MSE
Subsample: 1.0,   Max features: 230,   Max depth: 3

Residuals:
         Min           1Q       Median           3Q          Max
      -60.91       -1.597      -0.1425        1.388        21.06

            Pred_col Feature importance  Importance rank
           Intercept                 0.              230
 events_C...New Year                 0.               96
 events_C...w Year-1                 0.               95
 events_C...w Year-2                 0.               94
 events_C...w Year+1                 0.               93
 events_C...w Year+2                 0.               92
events_Christmas Day                 0.               91
 events_C...as Day-1                 0.               90
 events_C...as Day-2                 0.               89
 events_C...as Day+1   

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names



In [ ]:
#visualization
fig = result.forecast.plot_components()
iplot(fig)

In [ ]:
#visualization
fig = result.backtest.plot()
iplot(fig)

In [ ]:
#getting the forecast result
forecast = result.forecast.df[["Date", "forecast"]]
forecast = forecast.rename(columns = {'forecast': 'silverkite'})
predictions_silverkite = forecast.iloc[-31:,:]
predictions_silverkite

,Date,silverkite
13829,2023-11-13,80.390876
13830,2023-11-14,80.390876
13831,2023-11-15,80.056567
13832,2023-11-16,79.502800
13833,2023-11-17,78.785081
13834,2023-11-18,78.785081
13835,2023-11-19,79.091738
13836,2023-11-20,78.793818
13837,2023-11-21,78.793818
13838,2023-11-22,78.793818


In [ ]:
#exporting
predictions_silverkite.to_csv("/content/drive/MyDrive/PCDS_OilPricePrediction/forecasting_products/predictions_silverkite_linearGrowth.csv")

## Brent Data

### 1) Silverkite Model

In [ ]:
#Forecasting
forecaster_brent = Forecaster()
result_brent = forecaster_brent.run_forecast_config(df = eu_brent_fp,
                                        config = config_brent)

/usr/local/lib/python3.9/dist-packages/greykite/common/time_properties.py:197: UserWarning:

Dropped 1 dates when filling gaps in input data. Provide data frequency and make sure data points are evenly spaced.



Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.9/dist-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 29.84% of one input.

/usr/local/lib/python3.9/dist-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 31.82% of one input.

/usr/local/lib/python3.9/dist-packages/greykite/common/evaluation.py:153: UserWarning:

7 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.9/dist-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 31.82% of one input.

/usr/local/lib/python3.9/dist-packages/greykite/common/evaluation.py:153: UserWarning:

7 value(s) in y_true were NA or infinite and are omitted in error calc.

/usr/local/lib/python3.9/dist-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 31.82% of one in

In [ ]:
#Look at the model summary
summary_brent = result_brent.model[-1].summary() #-1 restrieves the estimator from the pipeline
print(summary_brent)

================================ Model Summary =================================

Number of observations: 13068,   Number of features: 182
Method: Gradient Boosting
Number of Trees: 100,   Criterion: FRIEDMAN_MSE
Subsample: 1.0,   Max features: 182,   Max depth: 3

Residuals:
         Min           1Q       Median           3Q          Max
      -18.93       -1.602        -0.12        1.352        20.32

            Pred_col Feature importance  Importance rank
           Intercept                 0.              182
 events_C...New Year                 0.               74
 events_C...w Year-1                 0.               73
 events_C...w Year-2                 0.               72
 events_C...w Year+1                 0.               71
 events_C...w Year+2                 0.               70
events_Christmas Day                 0.               69
 events_C...as Day-1                 0.               68
 events_C...as Day-2                 0.               67
 events_C...as Day+1  

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names



In [ ]:
#visualization
fig_brent = result_brent.forecast_brent.plot_components()
iplot(fig_brent)

In [ ]:
#visualization
fig1_brent = result_brent.backtest.plot()
iplot(fig1_brent)

In [ ]:
#getting the forecast result
forecast_brent = result_brent.forecast.df[["Date", "forecast"]]
forecast_brent = forecast_brent.rename(columns = {'forecast': 'silverkite'})
predictions_brent_silverkite = forecast_brent.iloc[-31:,:]
predictions_brent_silverkite

,Date,silverkite
13326,2023-11-13,85.913577
13327,2023-11-14,85.913577
13328,2023-11-15,85.913577
13329,2023-11-16,85.212256
13330,2023-11-17,85.212256
13331,2023-11-18,85.212256
13332,2023-11-19,85.212256
13333,2023-11-20,85.212256
13334,2023-11-21,85.212256
13335,2023-11-22,85.212256


In [ ]:
#exporting
predictions_brent_silverkite.to_csv("/content/drive/MyDrive/PCDS_OilPricePrediction/forecasting_products/predictions_brent_silverkite_linearGrowth.csv")

# 3_Parameter Tuning results

## Cushing Data

### 1) Silverkite Model

In [ ]:
#CV results
cv_results = summarize_grid_search_results(
    grid_search = result.grid_search,
    decimals = 1,
    score_func = EvaluationMetricEnum.RootMeanSquaredError.name)

In [ ]:
#Set the CV results index
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop = True, inplace = True)
cv_results

,rank_test_CORR,rank_test_R2,rank_test_MSE,rank_test_RMSE,rank_test_MAE,rank_test_MedAE,rank_test_MAPE,rank_test_MedAPE,rank_test_sMAPE,rank_test_Q80,...,split2_train_OutsideTolerance4p,std_train_OutsideTolerance4p,split0_test_OutsideTolerance5p,split1_test_OutsideTolerance5p,split2_test_OutsideTolerance5p,std_test_OutsideTolerance5p,split0_train_OutsideTolerance5p,split1_train_OutsideTolerance5p,split2_train_OutsideTolerance5p,std_train_OutsideTolerance5p
params,,,,,,,,,,,,,,,,,,,,,
"[('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",2,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.1,0.3,0.4,0.1,0.0,0.0,0.0,0.0
"[('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",1,2,2,2,2,2,2,2,2,2,...,0.2,0.1,0.9,0.7,0.4,0.2,0.1,0.2,0.1,0.1


In [ ]:
#Looking at the best results
cv_results[["rank_test_RMSE", "mean_test_RMSE","mean_test_MSE",
            "param_estimator__fit_algorithm_dict",
            "param_estimator__growth_term"]]

,rank_test_RMSE,mean_test_RMSE,mean_test_MSE,param_estimator__fit_algorithm_dict,param_estimator__growth_term
params,,,,,
"[('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",1,4.1,22.6,{'fit_algorithm': 'gradient_boosting'},linear
"[('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",2,5.8,46.3,{'fit_algorithm': 'ridge'},linear


In [ ]:
best_params = cv_results[cv_results.rank_test_RMSE == 1][["mean_test_RMSE",
                                            "param_estimator__fit_algorithm_dict",
                                            "param_estimator__growth_term"]].transpose()

In [ ]:
best_params.to_csv("/content/drive/MyDrive/PCDS_OilPricePrediction/forecasting_products/best_params_silverkite_linearGrowth.csv")

## Brent Data

### 1) Silverkite Model

In [ ]:
#CV results
cv_results_brent = summarize_grid_search_results(
    grid_search = result_brent.grid_search,
    decimals = 1,
    score_func = EvaluationMetricEnum.RootMeanSquaredError.name)

In [ ]:
#Set the CV results index
cv_results_brent["params"] = cv_results_brent["params"].astype(str)
cv_results_brent.set_index("params", drop = True, inplace = True)
cv_results_brent

,rank_test_CORR,rank_test_R2,rank_test_MSE,rank_test_RMSE,rank_test_MAE,rank_test_MedAE,rank_test_MAPE,rank_test_MedAPE,rank_test_sMAPE,rank_test_Q80,...,split2_train_OutsideTolerance4p,std_train_OutsideTolerance4p,split0_test_OutsideTolerance5p,split1_test_OutsideTolerance5p,split2_test_OutsideTolerance5p,std_test_OutsideTolerance5p,split0_train_OutsideTolerance5p,split1_train_OutsideTolerance5p,split2_train_OutsideTolerance5p,std_train_OutsideTolerance5p
params,,,,,,,,,,,,,,,,,,,,,
"[('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",2,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.8,0.3,0.5,0.2,0.0,0.0,0.0,0.0
"[('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",1,2,2,2,2,2,2,2,2,2,...,0.3,0.2,1.0,0.7,0.4,0.2,0.6,0.2,0.2,0.2


In [ ]:
#Looking at the best results
cv_results_brent[["rank_test_RMSE", "mean_test_RMSE","mean_test_MSE",
            "param_estimator__fit_algorithm_dict",
            "param_estimator__growth_term"]]

,rank_test_RMSE,mean_test_RMSE,mean_test_MSE,param_estimator__fit_algorithm_dict,param_estimator__growth_term
params,,,,,
"[('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",1,7.3,53.5,{'fit_algorithm': 'gradient_boosting'},linear
"[('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",2,11.3,150.1,{'fit_algorithm': 'ridge'},linear


In [ ]:
best_params_brent = cv_results_brent[cv_results_brent.rank_test_RMSE == 1][["mean_test_RMSE",
                                            "param_estimator__fit_algorithm_dict",
                                            "param_estimator__growth_term"]].transpose()

In [ ]:
best_params_brent.to_csv("/content/drive/MyDrive/PCDS_OilPricePrediction/forecasting_products/best_params_brent_silverkite_linearGrowth.csv")